In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
import struct as st


Mounted at /content/drive


In [ ]:
def read_data(filename):
    file = gzip.open(filename,'rb')
    file.seek(0)
    magic_number = st.unpack('>4B',file.read(4)) # read magic number
    number_of_images = st.unpack('>I',file.read(4))[0] # read number of images
    number_of_rows = st.unpack('>I',file.read(4))[0] #read number of rows
    number_of_columns = st.unpack('>I',file.read(4))[0] #read number of column
    with gzip.open(filename) as file_stream:
        file_stream.read(16)
        buf = file_stream.read(number_of_rows * number_of_columns * number_of_images)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(number_of_images, number_of_rows,number_of_columns)
        return data

In [ ]:
train_data = read_data('/content/train-images-idx3-ubyte.gz')
test_data = read_data('/content/t10k-images-idx3-ubyte.gz')

In [ ]:
def read_labels(filename):
    file = gzip.open(filename,'rb')
    file.seek(0)
    magic_number = st.unpack('>4B',file.read(4)) # read magic number
    number_of_iteams = st.unpack('>I',file.read(4))[0] # read number of images
    with gzip.open(filename) as file_stream:
        file_stream.read(8)
        buf = file_stream.read(number_of_iteams)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        return labels

In [ ]:
train_labels = read_labels('/content/train-labels-idx1-ubyte.gz')
test_labels = read_labels('/content/t10k-labels-idx1-ubyte.gz')

In [ ]:
# Create dictionary of target classes
label_dict = {
 0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
}

In [ ]:
train_data = train_data.reshape(-1, 28,28, 1)
test_data = test_data.reshape(-1, 28,28, 1)
train_data.shape, test_data.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [ ]:
train_data = train_data / np.max(train_data)
test_data = test_data / np.max(test_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(train_data,
                                                             train_data,
                                                             test_size=0.2,
                                                             random_state=13)

In [ ]:
batch_size = 128
epochs = 25
inChannel = 1
x, y = 28, 28
input_img = Input(shape = (x, y, inChannel))
num_classes = 10
n_inputs = train_data.shape[1]

In [ ]:

#encoder
#input = 28 x 28 x 1 (wide and thin)
conv1 = Conv2D(32, (3, 3), activation='softmax', padding='same')(input_img) #28 x 28 x 32
conv1 = BatchNormalization()(conv1)
conv1 = Conv2D(32, (3, 3), activation='softmax', padding='same')(conv1)
conv1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
conv2 = Conv2D(64, (3, 3), activation='softmax', padding='same')(pool1) #14 x 14 x 64
conv2 = BatchNormalization()(conv2)
conv2 = Conv2D(64, (3, 3), activation='softmax', padding='same')(conv2)
conv2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
conv3 = Conv2D(128, (3, 3), activation='softmax', padding='same')(pool2) #7 x 7 x 128 (small and thick)
conv3 = BatchNormalization()(conv3)
conv3 = Conv2D(128, (3, 3), activation='softmax', padding='same')(conv3)
conv3 = BatchNormalization()(conv3)
conv4 = Conv2D(256, (3, 3), activation='softmax', padding='same')(conv3) #7 x 7 x 256 (small and thick)
conv4 = BatchNormalization()(conv4)
conv4 = Conv2D(256, (3, 3), activation='softmax', padding='same')(conv4)
conv4 = BatchNormalization()(conv4)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(conv4)

#decoder
conv5 = Conv2D(256, (3, 3), activation='softmax', padding='same')(bottleneck) #7 x 7 x 128
conv5 = BatchNormalization()(conv5)
conv5 = Conv2D(256, (3, 3), activation='softmax', padding='same')(conv5)
conv5 = BatchNormalization()(conv5)
conv6 = Conv2D(128, (3, 3), activation='softmax', padding='same')(conv5) #7 x 7 x 64
conv6 = BatchNormalization()(conv6)
conv6 = Conv2D(128, (3, 3), activation='softmax', padding='same')(conv6)
conv6 = BatchNormalization()(conv6)
up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
conv7 = Conv2D(64, (3, 3), activation='softmax', padding='same')(up1) # 14 x 14 x 32
conv7 = BatchNormalization()(conv7)
conv7 = Conv2D(64, (3, 3), activation='softmax', padding='same')(conv7)
conv7 = BatchNormalization()(conv7)
    
up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())

In [ ]:
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))

Epoch 1/25
375/375 [==============================] - 21s 57ms/step - loss: 0.0508 - val_loss: 0.1085
Epoch 2/25
375/375 [==============================] - 21s 56ms/step - loss: 0.0138 - val_loss: 0.0783
Epoch 3/25
375/375 [==============================] - 21s 57ms/step - loss: 0.0086 - val_loss: 0.0131
Epoch 4/25
375/375 [==============================] - 21s 57ms/step - loss: 0.0064 - val_loss: 0.0080
Epoch 5/25
375/375 [==============================] - 22s 58ms/step - loss: 0.0052 - val_loss: 0.0048
Epoch 6/25
375/375 [==============================] - 22s 58ms/step - loss: 0.0044 - val_loss: 0.0033
Epoch 7/25
375/375 [==============================] - 21s 57ms/step - loss: 0.0039 - val_loss: 0.0032
Epoch 8/25
375/375 [==============================] - 21s 57ms/step - loss: 0.0035 - val_loss: 0.0029
Epoch 9/25
375/375 [==============================] - 22s 58ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 10/25
375/375 [==============================] - 21s 57ms/step - loss: 0.002

In [ ]:
# define an encoder model (without the decoder)
encoder = Model(inputs=input_img, outputs=bottleneck)

In [ ]:
# encode the train data
X_train_encode = encoder.predict(train_X)

In [ ]:
print(len(X_train_encode[10]))

7


In [ ]:
print(len(train_X[10]))

28


Na filaksoume tis nees eikones se file gia na tis diavasoume meso tis c++